In [1]:
import numpy as np
import pandas as pd
import datetime
import os
from tqdm import tqdm
from dateutil.parser import parse
from datetime import timedelta, datetime

In [2]:
data_path = './data/'

In [3]:
def date_add_hours(start_date, hours):
    end_date = parse(start_date) + timedelta(hours=hours)
    end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
    return end_date

In [4]:
aq = pd.read_hdf(data_path+'aq.hdf')

In [5]:
aq.shape

(377265, 8)

In [6]:
aq.head()

,station_id,time,PM2.5,PM10,NO2,CO,O3,SO2
0,aotizhongxin,2017-01-01 14:00:00,453.0,467.0,156.0,7.2,3.0,9.0
1,aotizhongxin,2017-01-01 15:00:00,417.0,443.0,143.0,6.8,2.0,8.0
2,aotizhongxin,2017-01-01 16:00:00,395.0,467.0,141.0,6.9,3.0,8.0
3,aotizhongxin,2017-01-01 17:00:00,420.0,484.0,139.0,7.4,3.0,9.0
4,aotizhongxin,2017-01-01 18:00:00,453.0,520.0,157.0,7.6,4.0,9.0


In [7]:
aq.isnull().any()

station_id    False
time          False
PM2.5          True
PM10           True
NO2            True
CO             True
O3             True
SO2            True
dtype: bool

In [10]:
for i, column_name in enumerate(aq.columns):
    print(column_name, ':', aq[column_name].isnull().sum(), 'missing')

station_id : 0 missing
time : 0 missing
PM2.5 : 24069 missing
PM10 : 99665 missing
NO2 : 22172 missing
CO : 46610 missing
O3 : 24337 missing
SO2 : 22102 missing


there are missing values for PM2.5, PM10, O3

In [8]:
stations = set(aq['station_id'])

In [9]:
len(stations)

35

there are 35 aq stations in total

In [10]:
aq_station_locations = pd.read_csv(data_path + "station_info.csv")

In [11]:
aq_station_locations.head()

,station_id,longitude,latitude
0,dongsi,116.417,39.929
1,tiantan,116.407,39.886
2,guanyuan,116.339,39.929
3,wanshouxigong,116.352,39.878
4,aotizhongxin,116.397,39.982


In [12]:
# get distance for each aq station
for index_t in aq_station_locations.index:
    row_t = aq_station_locations.loc[index_t]
    # location of target station
    long_t = row_t["longitude"]
    lati_t = row_t["latitude"]
    # station id
    station_id = row_t["station_id"]
    
    # get the distance of the target station to other stations
    all_dis = []
    for index in aq_station_locations.index:
        row = aq_station_locations.loc[index]
        long = row['longitude']
        lati = row['latitude']
        dis = np.sqrt((long-long_t)**2 + (lati-lati_t)**2)
        all_dis.append(dis)
    
    aq_station_locations[station_id] = all_dis

In [12]:
# aq_station_locations

In [13]:
# sort by distance
near_stations = {}
for index_t in aq_station_locations.index:
    t_station_id = aq_station_locations.loc[index_t]['station_id']
    ordered_stations_id = aq_station_locations.sort_values(by=t_station_id)['station_id'].values[1:]
    near_stations[t_station_id] = ordered_stations_id

In [14]:
# near_stations['dingling']

In [14]:
all_time = set(aq['time'])
len(all_time)

10779

there are 10779 hours in real data

In [15]:
min_time = aq['time'].min()
max_time = aq['time'].max()
min_time, max_time

('2017-01-01 14:00:00', '2018-04-30 23:00:00')

In [16]:
hours_should = len(pd.date_range(min_time, max_time, freq='H'))
hours_should

11626

there should be 12626 consecutive hours

In [17]:
hours_should - len(all_time)

847

847 hours are missing

fill missing value with the value of its nearest station in a specific hour

if nearest is missing, find the second nearest and so on

let's first make a list of all missing hours

In [18]:
time = min_time
missing_hours = []
while time <=  max_time:
    if time not in all_time:
        missing_hours.append(time)
    time = date_add_hours(time,1)

In [19]:
len(missing_hours)

847

In [20]:
missing_hours[-20:]

['2018-04-19 16:00:00',
 '2018-04-19 17:00:00',
 '2018-04-20 22:00:00',
 '2018-04-21 03:00:00',
 '2018-04-21 11:00:00',
 '2018-04-22 01:00:00',
 '2018-04-22 20:00:00',
 '2018-04-22 22:00:00',
 '2018-04-22 23:00:00',
 '2018-04-24 09:00:00',
 '2018-04-24 10:00:00',
 '2018-04-25 08:00:00',
 '2018-04-26 14:00:00',
 '2018-04-27 02:00:00',
 '2018-04-28 05:00:00',
 '2018-04-29 00:00:00',
 '2018-04-29 16:00:00',
 '2018-04-29 17:00:00',
 '2018-04-30 11:00:00',
 '2018-04-30 18:00:00']

missing value at start time can be filled by the value of nearby station, and after that there is no missing value at start time for all stations, so we can always fill missing value with the value of former hour

there are three cases:
1. for a specific hour, some stations have missing value
2. for a specific hour, all stations have missing value
3. a specific hour is missing for all stations

divide filling process into three steps:
1. for case 1, fill with nearby station
2. for case 2, since no station has missing value in previous hour, fill with previous hour value, step 1 and step 2 can be done together
3. for case 3, after step 1 and step 2, fill value with previous hour

In [24]:
aq.set_index(['station_id','time'], inplace=True)
aq.head()

PM2.5   PM10    NO2   CO   O3  SO2
station_id   time                                                   
aotizhongxin 2017-01-01 14:00:00  453.0  467.0  156.0  7.2  3.0  9.0
             2017-01-01 15:00:00  417.0  443.0  143.0  6.8  2.0  8.0
             2017-01-01 16:00:00  395.0  467.0  141.0  6.9  3.0  8.0
             2017-01-01 17:00:00  420.0  484.0  139.0  7.4  3.0  9.0
             2017-01-01 18:00:00  453.0  520.0  157.0  7.6  4.0  9.0

In [25]:
def fill_missing_nearby(station_id, time, label, near_stations):
    if (station_id, time) not in aq.index:
        return fill_missing_nearby(station_id, date_add_hours(time,-1), label, near_stations)
    near_station = near_stations[station_id]
    for station in near_station:
        row = aq.loc[(station,time)]
        if not pd.isnull(row[label]):
            return row[label]
    return fill_missing_nearby(station_id, date_add_hours(time,-1), label, near_stations)

In [26]:
for station_id, time in aq.index:
    row = aq.loc[(station_id,time)]
    for label in ['PM2.5','PM10','O3']:                
        if pd.isnull(row[label]):
            aq.loc[(station_id,time), label] = fill_missing_nearby(station_id, time, label, near_stations)      

In [27]:
aq_filled = aq.copy()
aq_filled.reset_index(inplace=True)

In [28]:
aq_filled.shape

(377265, 8)

In [29]:
aq_filled.isnull().any()

station_id    False
time          False
PM2.5         False
PM10          False
NO2            True
CO             True
O3            False
SO2            True
dtype: bool

there is no missing value in the data now. Next, fix missing hour problem

In [30]:
aq_filled.to_hdf(data_path+'aq_filled.hdf','w', complib='blosc', complevel=5)

Now fill in missing hours

In [22]:
aq = pd.read_hdf(data_path+'aq_filled.hdf')
aq.head()

,station_id,time,PM2.5,PM10,NO2,CO,O3,SO2
0,aotizhongxin,2017-01-01 14:00:00,453.0,467.0,156.0,7.2,3.0,9.0
1,aotizhongxin,2017-01-01 15:00:00,417.0,443.0,143.0,6.8,2.0,8.0
2,aotizhongxin,2017-01-01 16:00:00,395.0,467.0,141.0,6.9,3.0,8.0
3,aotizhongxin,2017-01-01 17:00:00,420.0,484.0,139.0,7.4,3.0,9.0
4,aotizhongxin,2017-01-01 18:00:00,453.0,520.0,157.0,7.6,4.0,9.0


In [23]:
aq.shape

(377265, 8)

In [24]:
aq.set_index(['station_id','time'], inplace=True)

In [120]:
# aq.loc[(slice(None),'2017-01-01 14:00:00'), :]

In [32]:
for i in tqdm(range(len(missing_hours))):
    re_hour = date_add_hours(missing_hours[i],-1)
# for hour in missing_hours:
#     pre_hour = date_add_hours(hour,-1)
    for station_id in stations:        
        aq.loc[(station_id, missing_hours[i]), :] = aq.loc[(station_id, pre_hour), :]
    print(missing_hours[i])

  0%|                                                                                          | 0/847 [00:00<?, ?it/s]

2017-01-23 15:00:00
2017-01-23 17:00:00
2017-01-24 00:00:00
2017-01-24 04:00:00
2017-01-24 09:00:00
2017-01-24 12:00:00
2017-01-24 13:00:00
2017-01-24 14:00:00


  1%|▊                                                                                 | 8/847 [00:00<00:11, 71.61it/s]

2017-01-24 15:00:00
2017-01-25 01:00:00
2017-01-25 06:00:00
2017-01-25 13:00:00
2017-01-25 14:00:00
2017-01-25 15:00:00
2017-01-25 16:00:00
2017-01-25 17:00:00


  2%|█▌                                                                               | 16/847 [00:00<00:11, 71.23it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████▊| 845/847 [4:07:13<00:46, 23.04s/it]

2018-04-30 11:00:00


100%|█████████████████████████████████████████████████████████████████████████████▉| 846/847 [4:07:37<00:23, 23.19s/it]

2018-04-30 18:00:00


100%|██████████████████████████████████████████████████████████████████████████████| 847/847 [4:08:00<00:00, 23.10s/it]


In [35]:
aq.reset_index(inplace=True)
aq.isnull().any()

station_id    False
time          False
PM2.5         False
PM10          False
NO2            True
CO             True
O3            False
SO2            True
dtype: bool

In [37]:
len(set(aq['time']))

11626

In [38]:
hours_should == len(set(aq['time']))

True

great. there is no missing hour now

In [39]:
aq.shape

(406910, 8)

In [40]:
aq.to_hdf(data_path+'aq_nomissinghour.hdf','w', complib='blosc', complevel=5)